# Cliente — Consulta del endpoint `/optimize`
Solicitud **POST** al servicio FastAPI que publica el modelo de optimización de producción y distribución.
> **Requisitos previos:** Tener el servicio corriendo:  
> `uvicorn app:app --reload --port 8000`

In [ ]:
# pip install requests pandas

In [1]:
import os, json, requests
import pandas as pd

In [7]:
# Configuración del endpoint (ajusta si desplegaste en otra URL/puerto)
ENDPOINT = os.environ.get("SNACK_OPTIMIZER_ENDPOINT", "http://localhost:8000/optimize")
print("Usando ENDPOINT:", ENDPOINT)

Usando ENDPOINT: http://localhost:8000/optimize


## Ejemplo

In [13]:
# Payload de ejemplo (demanda)
payload = {
    "dcs": [
        {"dc": "CD_Norte",  "demand_kg": 110000},
        {"dc": "CD_Centro", "demand_kg":  55000},
        {"dc": "CD_Sur",    "demand_kg":  35000}
    ]
}

In [4]:
print(json.dumps(payload, indent=2, ensure_ascii=False))

{
  "dcs": [
    {
      "dc": "CD_Norte",
      "demand_kg": 110000
    },
    {
      "dc": "CD_Centro",
      "demand_kg": 55000
    },
    {
      "dc": "CD_Sur",
      "demand_kg": 35000
    }
  ]
}


## Llamada al endpoint - resultados

- `produce`: producción por planta (kg)
- `ship`: embarques planta→CD (kg)
- `short`: faltantes por CD (kg)
- `total_cost` y `status` del solver

In [5]:
def call_endpoint(endpoint: str, payload: dict, timeout: int = 60):
    try:
        r = requests.post(endpoint, json=payload, timeout=timeout)
        r.raise_for_status()
        return r.json()
    except requests.exceptions.RequestException as e:
        # Mensaje útil para diagnóstico
        raise SystemExit(f"Error al llamar al endpoint: {e}\n"
                         f"Verifica que el servidor FastAPI esté ejecutándose y accesible en {endpoint}.")
        


In [14]:
res = call_endpoint(ENDPOINT, payload)
print("Status:", res.get("status"))
print("Costo total:", round(res.get("total_cost", 0.0), 2))

Status: Optimal
Costo total: 280301.0


In [15]:
df_produce = pd.DataFrame(res["produce"])
df_ship = pd.DataFrame(res["ship"])
df_short = pd.DataFrame(res["short"])

display(df_produce.head())
display(df_ship.head())
display(df_short.head())

,plant,produce_kg
0,PlantA,114000.0
1,PlantB,83700.0


,plant,dc,ship_kg
0,PlantA,CD_Norte,110000.0
1,PlantA,CD_Centro,4000.0
2,PlantB,CD_Centro,51000.0
3,PlantB,CD_Sur,32700.0


,dc,short_kg
0,CD_Norte,0.0
1,CD_Centro,0.0
2,CD_Sur,2300.0
